##Importing CSV File for Preprocessing

In [ ]:
import pandas as pd

data = pd.read_csv('final_lematize_translated_data.csv')

##Run all cells before preprocessing

In [ ]:
import nltk
import re
import json
import unicodedata

In [ ]:
!pip install emoji
!pip install demoji

In [ ]:
nltk.download([
...     "names",
...     "stopwords",
...     "state_union",
...     "averaged_perceptron_tagger",
...     "vader_lexicon",
...     "punkt",
        "wordnet",
... ])

In [ ]:
from gensim.models import Word2Vec
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from emoji import UNICODE_EMOJI

#PreProcessing Task


###Convert gujarati to english
###Convert all emojis to text
###Convert all text to lower case
###Remove all special characters or links or user tags
###Remove all stopwords
###Lemmatize words



In [ ]:
#Function to remove https and @ tags 
def remove_https_and_tags(data): 

  #Splitting data into words
  data = data.split()

  text = ''
  for word in data:
    
    #If word is not matched add it to string 
    if 'http' in word:
      # print(word)
      continue
    elif 'youtu' in word:
      continue
    else:
      text+=(word + ' ')

  #Removes extra spaces from starting and ending
  text.strip()

  return text

In [ ]:
#Function to convert emoji to text 
import emoji 

# add space near your emoji
def add_space(text):
  result = ''
  for word in emoji.get_emoji_regexp().split(text):
    result+=word
    result+=' '

  return result

def convert_emoji_to_text(data):

  data = data.split()

  #Before converting emojis to text , space is required between word and emoji if there is no space between them
  text = ''
  for word in data:
    text+=(add_space(word) + ' ')
   
  #Removes extra spaces from starting and ending 
  text = text.strip()

  #Now convert emoji to text
  text = emoji.demojize(text, delimiters=("", "")) 
   
  return text

In [ ]:
def remove_special_characters(data):

   #Converting all emojis to respective text
   text = convert_emoji_to_text(data)  

   #Keeping only alpha-numeric and space
   text = re.sub("[^(a-zA-Z \_)]", "", text)

   #Removes extra spaces
   text = re.sub(" +", " ", text)

   #Removing 'https links' and user tags '@*'
   text = remove_https_and_tags(text)

   #Removes spaces from start and end
   text = text.strip()

   #converting data to lower case
   text = text.lower()

   #Keeping only alpha-numeric and space
   text = re.sub("[^(a-zA-Z \_)]", "", text)
  
   return text

In [ ]:
def remove_stopwords(data):
  
  lemmatizer = WordNetLemmatizer()
  data = nltk.word_tokenize(data)
  text = ""
  for word in data:
     if word not in set(stopwords.words('english')):
       word = lemmatizer.lemmatize(word)
       text+= (word + " ")
  text = text.strip()

  return text

#Updating Data Code

##Previous method of converting list item to dictionary just for information

 msg = ast.literal_eval(msg)
      
      #finding first key,value pair
      dict_pairs = msg.items()
      pairs_iterator = iter(dict_pairs)
      first_pair = next(pairs_iterator)
      
      #key is first_pair[0]
      #value is first_pair[1]
      
      if(len(first_pair[1])):
        msg = first_pair[1][0]
        msg = remove_special_characters(msg)
        msg = remove_stopwords(msg)
        
        if(len(msg)):
          dict1 = {}
          list_msg = []
          list_msg.append(msg)
          dict1[first_pair[0]] = list_msg
          subcomment_data.append(dict1)

##Main Code

###Same task are repeated for message,comment,subcomment

##Preprocessing on CSV Data

In [ ]:
import json
import ast
for i in range(data.shape[0]):
# for i in range(4):

  print(i)
  #Message Section
  message = data['post_msg_final'][i]
  
  #Making commas in text as $, so that list can be easily split 
  j=1
  ans="["
  while(j<len(message)-1):
     if(message[j]=='}' and message[j+1]==','):
       ans+="},"
       j+=2
       continue
     if(message[j]==','):
       ans+='$'
     else:
       ans+=message[j]
     j+=1
  ans+="]"
  message = ans
  message = message.strip('][').split(", ")

  message_data = []

  #traversing list
  for msg in message:    
   
    if(len(msg)):

      ans = msg.strip('[{}]')
      
      #Breaking string into 2 parts id, message
      if ':' and '[' in ans:
        ans = ans.split(': [')
      if ':' in ans:
        ans = ans.split(':')
      if '[' in ans:    #this part isn't working 
        ans = ans.split('[')

      #ans[0] - id , ans[1] - message

      #Remving whitespaces from beginning and end from id
    #   print(ans[0])
      if ans!="VALUE":
        if ans[0][1]==' ':
          ans[0] = ans[0][:1]+ans[0][2:]
        if ans[0][-2]==' ':
          ans[0] = ans[0][:-2]+ans[0][-1]
        if(ans[0][18]!='_'):
          ans[0] = ans[0][:18] + '_'+ ans[0][18:]
    
      #If message exists
      if(1<len(ans) and len(ans[1])):
        msg = ans[1]
        msg = remove_special_characters(msg)
        msg = remove_stopwords(msg)

        if(len(msg)):
          dict1 = {}
          list_msg = []
          list_msg.append(msg)
          dict1[ans[0]] = list_msg
          message_data.append(dict1)
  
  # print(message_data)
  data['post_msg_tranlsated_final'][i] = message_data


  #Comments Section
  comments = data['post_comments_final'][i]

  #Modifying string where there is space btw { and ,
  ans1="["
  j=1
  while(j<len(comments)-2):
      if(comments[j]=='}' and comments[j+1]==' ' and comments[j+2]==','):
         ans1+="},"
         j+=3
         continue
      else:
        ans1+=comments[j]
      j+=1
  ans1+=comments[len(comments)-2]
  ans1+="]"
  comments = ans1

  #Making commas in text as $, so that list can be easily split 
  ans="["
  j=1
  while(j<len(comments)-1):
     if(comments[j]=='}' and comments[j+1]==','):
       ans+="},"
       j+=2
       continue
     if(comments[j]==','):
       ans+='$'
     else:
       ans+=comments[j]
     j+=1
  ans+="]"
  comments=ans
  comments = comments.strip('][').split(', ')

  #traversing list
  comment_data = []
  for msg in comments:
    
    if(len(msg)):
      
      ans = msg.strip('[{}]')


      if ':' and '[' in ans:
        ans = ans.split(': [')
      elif ':' in ans:
        ans = ans.split(':')
      elif '[' in ans:
        ans = ans.split('[')
      
    #   print(ans)      #    ---------
      if ans!="VALUE" and len(ans[0]) > 1:
        if ans[0][1]==' ':
          ans[0] = ans[0][:1]+ans[0][2:]
        if ans[0][-2]==' ':
          ans[0] = ans[0][:-2]+ans[0][-1]
    
      if(1<len(ans) and len(ans[1])):

        msg = ans[1]
        msg = remove_special_characters(msg)
        msg = remove_stopwords(msg)

        if(len(msg)):
          dict1 = {}
          list_msg = []
          list_msg.append(msg)
          dict1[ans[0]]=list_msg
          comment_data.append(dict1)


  # print(comment_data)
  data['post_comments_translated_final'][i] = comment_data



  #SubComments Section
  subcomments = data['post_subcomments_final'][i]
  ans1="["
  j=1
  while(j<len(subcomments)-2):
      if(subcomments[j]=='}' and subcomments[j+1]==' ' and subcomments[j+2]==','):
         ans1+="},"
         j+=3
         continue
      else:
        ans1+=subcomments[j]
      j+=1
  ans1+=subcomments[len(subcomments)-2]
  ans1+="]"
  subcomments = ans1
  j=1
  ans="["
  while(j<len(subcomments)-1):
     if(subcomments[j]=='}' and subcomments[j+1]==','):
       ans+="},"
       j+=2
       continue
     if(subcomments[j]==','):
       ans+='$'
     else:
       ans+=subcomments[j]
     j+=1
  ans+="]"
  subcomments = ans
  subcomments = subcomments.strip('][').split(', ')
   
  subcomment_data = []
  for msg in subcomments:

    if(len(msg)):

      ans = msg.strip('[{}]')
      
      if ':' and '[' in ans:
        ans = ans.split(': [')
      if ':' in ans:
        ans = ans.split(':')
      if '[' in ans:
        ans = ans.split('[')

      if ans!="VALUE" and len(ans[0]) > 1:
        if ans[0][1]==' ':
          ans[0] = ans[0][:1]+ans[0][2:]
        if ans[0][-2]==' ':
          ans[0] = ans[0][:-2]+ans[0][-1]

      if(1<len(ans) and len(ans[1])):
        msg = ans[1]
        msg = remove_special_characters(msg)
        msg = remove_stopwords(msg)
        
        if(len(msg)):
          dict1 = {}
          list_msg = []
          list_msg.append(msg)
          dict1[ans[0]] = list_msg
          subcomment_data.append(dict1)

  # print(subcomment_data) 
  data['post_subcomments_translated_final'][i] = subcomment_data


###Exporting pre-processed data to new file

In [ ]:
data.to_csv('final_lematize_translated_data.csv',index=False)

##Reading preprocessed data columns from CSV File

In [ ]:
import json
import ast

complete_msg= []

for i in range(data.shape[0]):

  print(i)
 
  #Message Section
  message = data['post_msg_tranlsated_final'][i]
  message = message.strip('][').split(", ")

  #traversing list
  for msg in message:    
   
    if(len(msg)):
      
      msg = ast.literal_eval(msg)      

      #finding first key,value pair
      dict_pairs = msg.items()
      pairs_iterator = iter(dict_pairs)
      first_pair = next(pairs_iterator)
    
      #key is first_pair[0]
      #value is first_pair[1]
    
      if(len(dict_pairs) and len(first_pair[1])):
        msg = first_pair[1][0]
        complete_msg.append(msg)
        



  #Comments Section
  comments = data['post_comments_translated_final'][i]
  comments = comments.strip('][').split(", ")

  #traversing list
  for msg in comments:
    
    if(len(msg)):

      msg = ast.literal_eval(msg)  
      #finding first key,value pair
      
      dict_pairs = msg.items()
      pairs_iterator = iter(dict_pairs)
      first_pair = next(pairs_iterator)
    
      #key is first_pair[0]
      #value is first_pair[1]
    
      if(len(dict_pairs) and len(first_pair[1])):
        msg = first_pair[1][0]
        complete_msg.append(msg)



  #SubComments Section
  subcomments = data['post_subcomments_translated_final'][i]
  subcomments = subcomments.strip('][').split(", ")

  for msg in subcomments:

    if(len(msg)):

      msg = ast.literal_eval(msg)  

      #finding first key,value pair
      dict_pairs = msg.items()
      pairs_iterator = iter(dict_pairs)
      first_pair = next(pairs_iterator)
    
      #key is first_pair[0]
      #value is first_pair[1]
    
      if(len(dict_pairs) and len(first_pair[1])):
        msg = first_pair[1][0]
        complete_msg.append(msg)

In [ ]:
sentences = complete_msg
for i in range(len(sentences)):
  print(i)
  sentences[i] = nltk.word_tokenize(sentences[i])
# print(sentences[0])

##Creating Word2vec model for given sentences

In [ ]:
model = Word2Vec(sentences,min_count=2)
words = model.wv.vocab
print(words)

##Code for creating and uploading dictionary

In [ ]:
import pandas as pd

data1 = pd.read_csv('word_dictionary.csv')

In [ ]:
word_dict = []
for key,item in words.items():
  temp = []
  temp.append(key)
  temp.append(item)
  word_dict.append(temp)

In [ ]:
colNames = data1.columns

data1_new = pd.DataFrame(data=word_dict, columns=colNames)

# concatenate old and new
df_complete = pd.concat([data1, data1_new], axis = 0)

In [ ]:
df_complete.to_csv('word_dictionary.csv',index=False)

##Code for creating and updating vector

In [ ]:
import pandas as pd

data2 = pd.read_csv('word_vector.csv')

In [ ]:
word_vec = []
for key,item in words.items():
  temp = []
  temp.append(key)
  temp.append(model.wv[key])
  word_vec.append(temp)

In [ ]:
colNames = data2.columns

data2_new = pd.DataFrame(data=word_vec, columns=colNames)

# concatenate old and new
df_complete1 = pd.concat([data2, data2_new], axis = 0)

In [ ]:
df_complete1.to_csv('word_vector.csv',index=False)

##Preprocessing data from text file

In [ ]:
file1 = open('data.txt', 'r')
# file2 = open('lemmatized_data.txt','w')
Lines = file1.readlines()

i=0
complete_data = []
for msg in Lines:
  print(i)
  msg = remove_special_characters(msg)
  msg = remove_stopwords(msg)
  complete_data.append(msg)
  i+=1

file1.close()
# file2.close()
  

##Tokenising sentence 

In [ ]:
sentences = complete_data
for i in range(len(sentences)):
  print(i)
  sentences[i] = nltk.word_tokenize(sentences[i])
  for word in sentences[i]:
    if len(word)<=3:
      sentences[i].remove(word)
# print(sentences[0])

##Creating Word2vec Model

In [ ]:
model = Word2Vec(sentences,min_count=1)
words = model.wv.vocab
print(words)